# CIFAR-10 Project

In [ ]:
import keras
from keras.layers import Input, Dense, Concatenate
from sklearn.model_selection import GridSearchCV, train_test_split
from scikeras.wrappers import KerasRegressor
from keras.models import Model
from keras.datasets import cifar10
import joblib
import numpy as np
import pandas as pd

threads = int(joblib.cpu_count() * 0.9)

## Preprocessing

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

x = np.vstack((x_train, x_test))
y = np.vstack((y_train, y_test))
y = y.flatten()

print(f"data shape: {x.shape}, {y.shape}")

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=7)
print(f"Training data shape: {x_train.shape}, {y_train.shape}")
print(f"Test data shape: {x_test.shape}, {y_test.shape}")

x_train_flat = x_train.reshape((x_train.shape[0], -1))
x_test_flat = x_test.reshape((x_test.shape[0], -1))

print(f"Flattened data shape: {x_train_flat.shape}, {x_test_flat.shape}")

In [ ]:
print(np.unique(y, return_counts=True))

input_layer = Input(shape=(3072,), name="input_layer")
hidden_layer_1 = Dense(512, activation="tanh", name="hidden_layer_1")(input_layer)
hidden_layer_2 = Dense(256, activation="relu", name="hidden_layer_2")(hidden_layer_1)
hidden_layer_3 = Dense(128, activation="relu", name="hidden_layer_3")(hidden_layer_2)
output_layer = Dense(10, activation="softmax", name="output_layer")(hidden_layer_3)

In [ ]:
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer="sgd", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=8),
    keras.callbacks.ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss"),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5)
]

model.fit(x_train_flat, y_train, validation_split=0.2, epochs=100, batch_size=32, callbacks=callbacks)

## Evaluate Model

In [ ]:
model.evaluate(x_test_flat, y_test, batch_size=32)